<a href="https://colab.research.google.com/github/poojavjpy/Data-Analytics/blob/main/YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive to access your dataset
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install ultralytics


In [ ]:
# Import necessary libraries
import os
from ultralytics import YOLO




# To clear the **cache**

In [ ]:
!rm /content/drive/MyDrive/locust_detection/dataset/roboflow/labels/train.cache

In [ ]:
!rm /content/drive/MyDrive/locust_detection/dataset/roboflow/labels/val.cache

# Dataset Loading

In [ ]:
# Define the paths to your dataset and where to save the model
dataset_path = '/content/drive/MyDrive...'  # Path to your YOLOv5 formatted dataset
save_model_path = '/content/drive/MyDrive/..'  # Path to save the trained YOLOv8 model

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # 'yolov8n.pt' is the nano version of YOLOv8. You can use 'yolov8s.pt', 'yolov8m.pt', etc. for different sizes


# Training

In [ ]:
# Define the correct path to your dataset's data.yaml
dataset_yaml_path = '/content/drive/MyDrive/locust_detection/.../robo.dataset.yaml'  # Ensure this is the correct path

# Train the YOLOv8 model using the correct path
results = model.train(
    data=dataset_yaml_path,  # Use the correct path here
    epochs=60,
    batch=16,
    imgsz=640,
    save_period=10,
    project=save_model_path,
    name='yolov8_detection'
)


# Checking Results

In [ ]:
import pandas as pd
from google.colab import drive
import os

# Mount Google Drive to access the results file
drive.mount('/content/drive')

def extract_metrics(file_path):
    # Read the CSV file from the provided file path
    try:
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.strip()  # Clean up column names

        # Print the column names for inspection
        print("Columns in the CSV file:", df.columns.tolist())

        # Extract precision, recall, and mAP metrics from the CSV file
        precision = df['metrics/precision(B)'].values[-1]  # Get last epoch's precision
        recall = df['metrics/recall(B)'].values[-1]        # Get last epoch's recall
        map50 = df['metrics/mAP50(B)'].values[-1]          # Get last epoch's mAP50
        map50_95 = df['metrics/mAP50-95(B)'].values[-1]    # Get last epoch's mAP50-95

        return precision, recall, map50, map50_95

    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return None  # Return None when the file is not found
    except KeyError as e:
        print(f"Error: Missing column {e} in the CSV file.")
        return None  # Return None if a column is missing
    except Exception as e:
        print(f"An error occurred: {e}")
        return None  # Return None for any other exception

# Provide the path to your YOLO results CSV file within Google Drive
file_path = '/content/drive/MyDrive/yolov8_model/robo yolo v8/yolov8_detection24/results.csv'  # Update with your actual file path

# Check if the file exists
if os.path.isfile(file_path):
    # Extract and display precision, recall, and mAP metrics
    metrics = extract_metrics(file_path)
    if metrics is None:
        print("Failed to extract metrics.")
    else:
        precision, recall, map50, map50_95 = metrics
        print(f"Precision: {precision * 100:.2f}%")
        print(f"Recall: {recall * 100:.2f}%")
        print(f"mAP@50: {map50 * 100:.2f}%")
        print(f"mAP@50-95: {map50_95 * 100:.2f}%")
else:
    print(f"Error: File not found at {file_path}. Please check the path.")


In [ ]:
from ultralytics import YOLO
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt

# Define paths
weights_path = '/content/drive/MyDrive/../weights/last.pt'  # Path to the trained model weights
source_directory = '/content/drive/MyDrive/locust_detection/test_data/images'  # Directory containing images
output_directory = '/content/drive/MyDrive/yolov8_output/robo yolo v8'  # Directory to save output images

# Load the YOLOv8 model
model = YOLO(weights_path)

# Perform inference on all images in the source directory
results = model.predict(source=source_directory, conf=0.40, save=True, project=output_directory, name='detection_results')

# Move images if YOLOv8 saves them in subfolders
for root, dirs, files in os.walk(output_directory):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            src_path = os.path.join(root, file)
            dst_path = os.path.join(output_directory, file)
            shutil.move(src_path, dst_path)

# Function to resize images
def resize_image(image_path, size=(640, 480)):
    with Image.open(image_path) as img:
        img = img.resize(size, Image.Resampling.LANCZOS)  # Updated from ANTIALIAS to LANCZOS
        return img

# Display resized images
image_files = [f for f in os.listdir(output_directory) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
num_images = len(image_files)

# Set grid size
cols = 5
rows = (num_images // cols) + (num_images % cols > 0)  # Number of rows needed

plt.figure(figsize=(cols * 4, rows * 4))  # Adjust the figure size based on the grid

for i, image_file in enumerate(image_files):
    img_path = os.path.join(output_directory, image_file)
    img = resize_image(img_path)

    plt.subplot(rows, cols, i + 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Image {i + 1}")

plt.tight_layout()
plt.show()
